# Phân loại bệnh cúm bằng Naive Bayes

In [ ]:
import numpy as np
import pandas as pd
data = pd.DataFrame({
    "Sốt": ["Cao", "Cao", "Cao", "Thấp", "Cao", "Thấp", "Thấp", "Cao", "Thấp", "Cao"],
    "Ho": ["Có", "Không", "Có", "Có", "Có", "Không", "Có", "Không", "Không", "Có"],
    "Đau họng": ["Có", "Có", "Không", "Có", "Có", "Có", "Không", "Có", "Không", "Có"],
    "Mệt mỏi": ["Có", "Không", "Có", "Có", "Không", "Không", "Có", "Có", "Không", "Có"],
    "Flu": ["Có", "Có", "Có", "Không", "Có", "Không", "Không", "Có", "Không", "Có"]
})

data


,Sốt,Ho,Đau họng,Mệt mỏi,Flu
0,Cao,Có,Có,Có,Có
1,Cao,Không,Có,Không,Có
2,Cao,Có,Không,Có,Có
3,Thấp,Có,Có,Có,Không
4,Cao,Có,Có,Không,Có
5,Thấp,Không,Có,Không,Không
6,Thấp,Có,Không,Có,Không
7,Cao,Không,Có,Có,Có
8,Thấp,Không,Không,Không,Không
9,Cao,Có,Có,Có,Có


In [ ]:
new_patient = {"Sốt": "Cao", "Ho": "Có", "Đau họng": "Có", "Mệt mỏi": "Không"}

P_flu = data["Flu"].value_counts(normalize = True)["Có"] # P(C =1)
#tính xác suất tiên nghiệm (prior probability) của việc mắc bệnh cúm
# xác suất của trường hợp "Có" (cúm) dựa trên dữ liệu
P_not_flu = 1 - P_flu # P(C=0)


def conditional_prob ( feature , value , flu_status ): #xác suất tiên nghiệm của lớp Flu, sau khi bổ sung Lap Smooth
    data_subset = data [ data["Flu"] == flu_status] #lọc dữ liệu theo cột Flu= có/không
    numerator = len(data_subset[data_subset[feature] ==value]) + 1 #Trong nhóm Flu = Có/không, số lần feature = value là len. +1 theo Laplace
    # số lượng mẫu trong tập con data_subset có giá trị của feature bằng value
    denominator = len(data_subset) +2 # +2: mỗi feature có 2 giá trị khả dĩ là có/không hoặc cặp cao/thấp # Tính tổng số mẫu trong tập con
    return numerator/denominator # Xác suất tiên nghiệm của lớp Flu

P_X_given_flu = 1  # xác suất có điều kiện. xác suất xảy ra sự kiện X khi biết rằng một người bị cúm
P_X_given_not_flu = 1

for feature, value in new_patient.items():  #Lặp qua từng đặc trưng của bệnh nhân mới.
# new_patient là một dictionary chứa thông tin của một bệnh nhân mới
# .items() giúp ta lấy từng cặp (feature, value) trong dictionary
    prob_given_flu = conditional_prob(feature, value, "Có")
    # Gọi hàm conditional_prob(feature, value, "Có") để tính P(feature=value∣Flu=Có)
    prob_given_not_flu = conditional_prob(feature, value, "Không")
    #ính xác suất có điều kiện của từng đặc trưng trong trường hợp có cúm/ không cúm

    P_X_given_flu *= prob_given_flu   #Cập nhật xác suất tổng thể
    P_X_given_not_flu *= prob_given_not_flu #Nhân dần vào tổng xác suất để áp dụng quy tắc của Naive Bayes

    print(f"P({feature}={value}|Flu=Có) = {prob_given_flu:.4f}") #in xác suất có điều kiện của từng đặc trưng
    print(f"P({feature}={value}|Flu=Không) = {prob_given_not_flu:.4f}")

# Tính P(C|X) trực tiếp từ tỷ lệ (không cần P(X)) # Xác suất hậu nghiệm
# tính tử số của công thức định lý Bayes cho từng trường hợp Flu = Có và Flu = Không.
numerator_flu = P_X_given_flu * P_flu
numerator_not_flu = P_X_given_not_flu * P_not_flu

# Chuẩn hóa xác suất
total = numerator_flu + numerator_not_flu # Xác suất xuất hiện triệu chứng khi có cúm và không cúm
P_flu_given_X = numerator_flu / total  #xác suất một người bị cúm khi có sự kiện X
P_not_flu_given_X = numerator_not_flu / total

print("\nKết quả Naive Bayes:")
print(f"P(Flu=Có|X) = {P_flu_given_X:.4f}")
print(f"P(Flu=Không|X) = {P_not_flu_given_X:.4f}")

# Kết luận
print("\nKết luận:", "Bệnh nhân có khả năng bị cúm." if P_flu_given_X > P_not_flu_given_X else "Bệnh nhân KHÔNG có khả năng bị cúm.")



P(Sốt=Cao|Flu=Có) = 0.8750
P(Sốt=Cao|Flu=Không) = 0.1667
P(Ho=Có|Flu=Có) = 0.6250
P(Ho=Có|Flu=Không) = 0.5000
P(Đau họng=Có|Flu=Có) = 0.7500
P(Đau họng=Có|Flu=Không) = 0.5000
P(Mệt mỏi=Không|Flu=Có) = 0.3750
P(Mệt mỏi=Không|Flu=Không) = 0.5000

Kết quả Naive Bayes:
P(Flu=Có|X) = 0.9172
P(Flu=Không|X) = 0.0828

Kết luận: Bệnh nhân có khả năng bị cúm.


In [ ]:
import numpy as np
import pandas as pd

# Dữ liệu huấn luyện
data = pd.DataFrame({
    "Sốt": ["Cao", "Cao", "Cao", "Thấp", "Cao", "Thấp", "Thấp", "Cao", "Thấp", "Cao"],
    "Ho": ["Có", "Không", "Có", "Có", "Có", "Không", "Có", "Không", "Không", "Có"],
    "Đau họng": ["Có", "Có", "Không", "Có", "Có", "Có", "Không", "Có", "Không", "Có"],
    "Mệt mỏi": ["Có", "Không", "Có", "Có", "Không", "Không", "Có", "Có", "Không", "Có"],
    "Flu": ["Có", "Có", "Có", "Không", "Có", "Không", "Không", "Có", "Không", "Có"]
})

# Dữ liệu bệnh nhân cần dự đoán
new_patient = {"Sốt": "Cao", "Ho": "Có", "Đau họng": "Có", "Mệt mỏi": "Không"}

# Tính xác suất tiên nghiệm P(C)
P_flu = data["Flu"].value_counts(normalize=True)["Có"]  # P(C=1)
P_not_flu = 1 - P_flu  # P(C=0)

# Hàm tính xác suất có điều kiện P(x_i|C) với Laplace smoothing
def conditional_prob(feature, value, flu_status):
    # Laplace smoothing thêm 1 vào tử số để tránh xác suất = 0
    # Ví dụ: Nếu không có trường hợp nào "Sốt=Cao" khi "Flu=Có" thì P(Sốt=Cao|Flu=Có) = 0
    # Điều này sẽ làm cho tích các xác suất = 0, gây mất thông tin
    subset = data[data["Flu"] == flu_status]
    count_feature = len(subset[subset[feature] == value]) + 1  # Thêm 1 vào tử số

    # Thêm 2 vào mẫu số vì mỗi đặc trưng có 2 giá trị (VD: Sốt: Cao/Thấp)
    # Để đảm bảo tổng xác suất = 1: P(Cao|Flu) + P(Thấp|Flu) = 1
    total_count = len(subset) + 2  # Thêm 2 vào mẫu số
    return count_feature / total_count

# Tính P(X|C) = P(x_1|C) * P(x_2|C) * ... * P(x_n|C) theo Naive Bayes
P_X_given_flu = 1  # P(X|C=1)
P_X_given_not_flu = 1  # P(X|C=0)

for feature, value in new_patient.items():
    prob_given_flu = conditional_prob(feature, value, "Có")
    prob_given_not_flu = conditional_prob(feature, value, "Không")

    P_X_given_flu *= prob_given_flu
    P_X_given_not_flu *= prob_given_not_flu

    print(f"P({feature}={value}|Flu=Có) = {prob_given_flu:.4f}")
    print(f"P({feature}={value}|Flu=Không) = {prob_given_not_flu:.4f}")

# Tính P(C|X) trực tiếp từ tỷ lệ (không cần P(X))
numerator_flu = P_X_given_flu * P_flu
numerator_not_flu = P_X_given_not_flu * P_not_flu

# Chuẩn hóa xác suất
total = numerator_flu + numerator_not_flu
P_flu_given_X = numerator_flu / total
P_not_flu_given_X = numerator_not_flu / total

print("\nKết quả Naive Bayes:")
print(f"P(Flu=Có|X) = {P_flu_given_X:.4f}")
print(f"P(Flu=Không|X) = {P_not_flu_given_X:.4f}")

# Kết luận
print("\nKết luận:", "Bệnh nhân có khả năng bị cúm." if P_flu_given_X > P_not_flu_given_X else "Bệnh nhân KHÔNG có khả năng bị cúm.")

P(Sốt=Cao|Flu=Có) = 0.8750
P(Sốt=Cao|Flu=Không) = 0.1667
P(Ho=Có|Flu=Có) = 0.6250
P(Ho=Có|Flu=Không) = 0.5000
P(Đau họng=Có|Flu=Có) = 0.7500
P(Đau họng=Có|Flu=Không) = 0.5000
P(Mệt mỏi=Không|Flu=Có) = 0.3750
P(Mệt mỏi=Không|Flu=Không) = 0.5000

Kết quả Naive Bayes:
P(Flu=Có|X) = 0.9172
P(Flu=Không|X) = 0.0828

Kết luận: Bệnh nhân có khả năng bị cúm.


In [ ]:
# prompt: Hãy viết code để giải bài toán Linear Regression với batch size =5, cho bộ dữ liệu tại /content/advertising.csv, không huấn luyện trên từng sample mà huấn luyện theo từng nhóm 5 sample. in kết quả ra weights và bias.

import numpy as np
import pandas as pd

# Load the dataset
data = pd.read_csv('/content/advertising.csv')
X = data[['TV', 'Radio', 'Newspaper']].values
y = data['Sales'].values

# Normalize the features
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# Add bias term
X = np.c_[np.ones(X.shape[0]), X]

# Hyperparameters
learning_rate = 0.01
n_iterations = 1000
batch_size = 5

# Initialize weights and bias
n_samples, n_features = X.shape
weights = np.zeros(n_features)

# Mini-batch gradient descent
for _ in range(n_iterations):
    for i in range(0, n_samples, batch_size):
        X_batch = X[i:i + batch_size]
        y_batch = y[i:i + batch_size]

        # Calculate predictions
        y_predicted = np.dot(X_batch, weights)

        # Calculate gradients
        dw = (1 / batch_size) * np.dot(X_batch.T, (y_predicted - y_batch))

        # Update weights
        weights = weights - learning_rate * dw

# Print the results
print("Weights:", weights)
print("Bias:", weights[0])


Weights: [ 1.51288129e+01  4.67260051e+00  1.59579180e+00 -1.16493326e-02]
Bias: 15.128812905336783
